In [1]:
from utils.get_data import generate_data_nested_dirs, get_cube_at_point, generate_cubes, generate_cube_batch, nested_dirs
import numpy as np
import matplotlib.pylab as plt
from SimpleITK import ReadImage, GetArrayFromImage, sitkFloat32
from random import shuffle
import os

In [2]:
import pandas as pd

df_annotations = pd.read_csv('CSVFILES/annotations.csv')

In [ ]:
def plot_one_scan(scan:np.array):
    plt.figure(figsize=(32,128))
    plt.gray()
    plt.subplots_adjust(0,0,0.5,0.5,0.01,0.01)
    for i in range(scan.shape[0]):
        shape0 = scan.shape[0] // 4 + 1
        plt.subplot(shape0, 4, i + 1)
        plt.imshow(scan[i])
        plt.axis('off')
    plt.show()

In [ ]:
scans = generate_data_nested_dirs('./data')
scan = next(scans)[0]
plot_one_scan(scan)

In [ ]:
def scan_images(scan:np.array, filename='scan_image', sep='_'):
    for i in range(scan.shape[0]):
        plt.gray()
        plt.figure(figsize=(10,10))
        plt.imshow(scan[i])
        plt.axis('off')
        plt.savefig(f'{filename}{sep}{i}.png', bbox_inches='tight', pad_inches=0, transparent=True)

In [20]:
@nested_dirs
def generate_filenames(dirname = '.'):
    files=filter(lambda x: x[-3:] == 'mhd', os.listdir(dirname))
    for filename in files:
        yield dirname,filename[:-4]

In [22]:
files = generate_filenames('./data')
files=list(files)
df_files = pd.DataFrame(files, columns=['path','filename'])
df_files

,path,filename
0,./data/test,1.3.6.1.4.1.14519.5.2.1.6279.6001.162207236104...
1,./data/test,1.3.6.1.4.1.14519.5.2.1.6279.6001.162351539386...
2,./data/test,1.3.6.1.4.1.14519.5.2.1.6279.6001.162718361851...
3,./data/test,1.3.6.1.4.1.14519.5.2.1.6279.6001.162845309248...
4,./data/test,1.3.6.1.4.1.14519.5.2.1.6279.6001.162901839201...
...,...,...
883,./data/val,1.3.6.1.4.1.14519.5.2.1.6279.6001.161067514225...
884,./data/val,1.3.6.1.4.1.14519.5.2.1.6279.6001.161073793312...
885,./data/val,1.3.6.1.4.1.14519.5.2.1.6279.6001.161633200801...
886,./data/val,1.3.6.1.4.1.14519.5.2.1.6279.6001.161821150841...


In [ ]:
test_batch = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/test',
    batch_size = 32
)
train_batch = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/train',
    batch_size = 32
)
val_batch = generate_cube_batch(
    locations = df_annotations,
    target_size = (25,64,64),
    root_dir = 'data/val',
    batch_size = 32
)

def cubes_from_batch(batch_gen):
    for arr, batch in batch_gen:
        batch = list(batch)
        shuffle(batch)
        for result in batch:
            yield result

test_generator = cubes_from_batch(test_batch)
train_generator = cubes_from_batch(train_batch)
val_generator = cubes_from_batch(val_batch)

i = 0
while True:
    i += 1
    
    print('Test:',next(test_generator)[1], 'Train:',next(train_generator)[1], 'Val: ',next(val_generator)[1],sep=' ')
    